To learn more about accelerating pandas on Colab, see the [10 minute guide](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb) or
 [US stock market data analysis demo](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_stocks_demo.ipynb).

<div class="markdown-google-sans">

## Machine learning
</div>

With Colab you can import an image dataset, train an image classifier on it, and evaluate the model, all in just [a few lines of code](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb).

In [ ]:
# from CRITIC
!pip install func_timeout
import func_timeout


def safe_execute(code_string: str, keys=None):
    def execute(x):
        try:
            exec(x)
            locals_ = locals()
            if keys is None:
                an = locals_.get('answer', None)
            else:
                an = [locals_.get(k, None) for k in keys]
            return an, "Done"
        except BaseException as e: # jump wrong case
            return None, repr(e)

    try:
        an, report = func_timeout.func_timeout(3, execute, args=(code_string,))
    except func_timeout.FunctionTimedOut:
        an = None
        report = "TimeoutError: execution timeout"

    return an, report


def _test_safe_excute():
    code_string_1 = """import numpy as np
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
an = np.array([4, 5, 6])
answer = a + c"""

    code_string_2 = """budget = 1000
food = 0.3
accommodation = 0.15
entertainment = 0.25
coursework_materials = 1 - food - accommodation - entertainment
answer = budget * coursework_materials
"""
    an, report = safe_execute(code_string_1)
    print(an, report)

    an, report = safe_execute(code_string_2)
    print(an, report)


if __name__ == "__main__":
    _test_safe_excute()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for func_timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15077 sha256=fd901f1265c3be59bfb0630c47d3425319c15b5932336384983fe4aa684d7d00
  Stored in directory: /root/.cache/pip/wheels/3f/83/19/b5552bb9630e353f7c5b15be44bf10900afe1abbbfcf536afd
Successfully built func_timeout
None NameError("name 'c' is not defined")
299.99999999999994 Done


In [ ]:
class API:
    def check_api_call_correctness(self, response, groundtruth) -> bool:
        """
        Checks if the response from the API call is correct.

        Parameters:
        - response (dict): the response from the API call.
        - groundtruth (dict): the groundtruth response.

        Returns:
        - is_correct (bool): whether the response is correct.
        """
        raise NotImplementedError
    def call(self, **kwargs) -> dict:
        """
        Calls the API with the given parameters.

        Parameters:
        - kwargs (dict): the parameters to call the API with.

        Returns:
        - response (dict): the response from the API call.
        """
        raise NotImplementedError

class Calculator(API):
    description = 'This API provides basic arithmetic operations: addition, subtraction, multiplication, and division.'
    input_parameters = {
        'formula': {'type': 'str', 'description': 'The formula that needs to be calculated. Only integers are supported. Valid operators are +, -, *, /, and (, ). For example, \'(1 + 2) * 3\'.'},
    }
    output_parameters = {
        'result': {'type': 'float', 'description': 'The result of the formula.'},
    }
    def __init__(self) -> None:
        pass

    def check_api_call_correctness(self, response, groundtruth) -> bool:
        """
        Checks if the response from the API call is correct.

        Parameters:
        - response (dict): the response from the API call.
        - groundtruth (dict): the groundtruth response.

        Returns:
        - is_correct (bool): whether the response is correct.
        """
        re_formula = response['input']['formula'].replace(' ', '')
        gt_formula = groundtruth['input']['formula'].replace(' ', '')

        if re_formula == gt_formula and response['output'] == groundtruth['output'] and response['exception'] == groundtruth['exception']:
            return True
        else:
            return False

    def call(self, formula: str) -> float:
        """
        Calculates the result of the formula.

        Parameters:
        - formula (str): the formula that needs to be calculated. Valid operators are +, -, *, /, and (, ). For example, '(1 + 2) * 3'.

        Returns:
        - result (float): the result of the formula.
        - formula (str): the formula that was calculated.
        """
        input_parameters = {
            'formula': formula,
        }
        try:
            result = self.calculate(formula)
        except Exception as e:
            exception = str(e)
            return {'api_name': self.__class__.__name__, 'input': input_parameters, 'output': None, 'exception': exception}
        else:
            return {'api_name': self.__class__.__name__, 'input': input_parameters, 'output': result, 'exception': None}

    def calculate(self, formula: str) -> float:
        """
        Calculates the result of the formula.

        Parameters:
        - formula (str): the formula that needs to be calculated. Valid operators are +, -, *, /, and (, ). For example, '(1 + 2) * 3'.

        Returns:
        - result (float): the result of the formula.
        """
        # Remove all spaces from the formula
        formula = formula.replace(' ', '')

        # Check if the formula is valid
        if not self.is_valid_formula(formula):
            raise Exception('invalid formula')

        # Convert the formula to a list
        formula = self.convert_formula_to_list(formula)

        # Calculate the result
        result = self.calculate_formula(formula)

        return result

    def is_valid_formula(self, formula: str) -> bool:
        """
        Checks if the formula is valid.

        Parameters:
        - formula (str): the formula that needs to be checked.

        Returns:
        - is_valid (bool): True if the formula is valid, False otherwise.
        """
        # Check if the formula is empty
        if len(formula) == 0:
            return False

        # Check if the formula contains invalid characters
        dot_count = 0
        for c in formula:
            if c == '.':
                dot_count += 1
                if dot_count > 1:
                    return False
            else:
                dot_count = 0
            if c not in '0123456789+-*/().':
                return False

        # Check if the formula contains an invalid number of parentheses
        if formula.count('(') != formula.count(')'):
            return False

        # Check if the formula contains an invalid number of operators
        if formula.count('+') + formula.count('-') + formula.count('*') + formula.count('/') == 0:
            return False

        # Check if the formula contains an invalid number of operands
        if formula.count('+') + formula.count('-') + formula.count('*') + formula.count('/') + 1 == len(formula):
            return False

        return True

    def convert_formula_to_list(self, formula: str) -> list:
        """
        Converts the formula to a list.

        Parameters:
        - formula (str): the formula that needs to be converted.

        Returns:
        - formula_list (list): the formula converted to a list.
        """
        formula_list = []
        number = ''
        for c in formula:
            if c in '0123456789.':
                number += c
            else:
                if number != '':
                    formula_list.append(float(number))
                    number = ''
                formula_list.append(c)
        if number != '':
            formula_list.append(float(number))

        return formula_list

    def calculate_formula(self, formula: list) -> float:
        """
        Calculates the result of the formula.

        Parameters:
        - formula (list): the formula that needs to be calculated.

        Returns:
        - result (float): the result of the formula.
        """
        # Calculate the result of the parentheses
        while '(' in formula:
            left_parenthesis_index = formula.index('(')
            right_parenthesis_index = formula.index(')')
            formula[left_parenthesis_index:right_parenthesis_index + 1] = [self.calculate_formula(formula[left_parenthesis_index + 1:right_parenthesis_index])]

        # Calculate the result of the multiplication and division
        while '*' in formula or '/' in formula:
            if '*' in formula and '/' in formula:
                if formula.index('*') < formula.index('/'):
                    index = formula.index('*')
                else:
                    index = formula.index('/')
            elif '*' in formula:
                index = formula.index('*')
            else:
                index = formula.index('/')
            formula[index - 1:index + 2] = [self.calculate_operation(formula[index - 1], formula[index], formula[index + 1])]

        # Calculate the result of the addition and subtraction
        while '+' in formula or '-' in formula:
            if '+' in formula and '-' in formula:
                if formula.index('+') < formula.index('-'):
                    index = formula.index('+')
                else:
                    index = formula.index('-')
            elif '+' in formula:
                index = formula.index('+')
            else:
                index = formula.index('-')
            formula[index - 1:index + 2] = [self.calculate_operation(formula[index - 1], formula[index], formula[index + 1])]

        return formula[0]

    def calculate_operation(self, operand1: float, operator: str, operand2: float) -> float:
        """
        Calculates the result of the operation.

        Parameters:
        - operand1 (float): the first operand.
        - operator (str): the operator.
        - operand2 (float): the second operand.

        Returns:
        - result (float): the result of the operation.
        """
        if operator == '+':
            return operand1 + operand2
        elif operator == '-':
            return operand1 - operand2
        elif operator == '*':
            return operand1 * operand2
        elif operator == '/':
            return operand1 / operand2

# if __name__ == '__main__':
#     # Create the API
#     api = Calculator()
#     response = api.call('(1+2)* 3')
#     print(response["output"])

In [ ]:
!pip install httpcore
!pip install httpcore==0.15.0 httpx pymongo googletrans
!pip install openai

from googletrans import Translator
import openai
import time
import spacy
from googleapiclient.discovery import build
from sympy import sympify
import datetime

cx = ""
keys = [""]
openai.api_key = ""
openai.api_base = ""

def split_text(text):
    """
    使用 spaCy 进行句子分割。

    参数:
    text (str): 要分割的文本。

    返回:
    list: 分割后的句子列表。
    """
    # 加载英文模型（确保您已经安装并下载了该模型）
    nlp = spacy.load("en_core_web_sm")
    # 使用 spaCy 处理文本
    doc = nlp(text)

    # 提取句子
    sentences = [sentence.text.strip() for sentence in doc.sents]

    return sentences


def web_search(sentence, api_keys=keys, lr='lang_en', num=5):
    for api_key in api_keys:
        try:
            service = build("customsearch", "v1", developerKey=api_key)
            res = (
                service.cse()
                .list(
                    q=sentence,
                    cx=cx,
                    lr=lr,
                    num=num
                )
                .execute()
            )
            snippets = []
            if 'items' in res:
                for result in res['items']:
                    if "snippet" in result:
                        snippets.append(result["snippet"])
                fact = "\n".join(snippets)
                return fact  # 如果请求成功，返回结果并退出函数
            else:
                return "No results found."  # 如果没有找到结果，返回相应信息
        except Exception as e:
            print(f"An error occurred:\n{e}")

def match(sentence, context):
    template = "Given a piece of text and its corresponding context, please determine whether the text contains incorrect content. If you think there is an error, answer \"Yes\". Otherwise, answer \"No\".\n\n#Given Text#: {sentence}\n#Context#: {context}\n#Your Judgment#:\n\n#Given Text#: {sentence}\n#Context#: {context}\n#Your Judgment#:"
    input = template.format(sentence=sentence, context=context)
    print(input)
    message = [
        {"role": "user", "content": input},
    ]
    while True:
        try:
            res = openai.ChatCompletion.create(
                # model="gpt-3.5-turbo",
                model="gpt-4-turbo",
                messages=message,
                temperature=0,
            )
            break
        except openai.error.RateLimitError:
            print('openai.error.RateLimitError\nRetrying...')
            time.sleep(20)
        except openai.error.ServiceUnavailableError:
            print('openai.error.ServiceUnavailableError\nRetrying...')
            time.sleep(20)
        except openai.error.Timeout:
            print('openai.error.Timeout\nRetrying...')
            time.sleep(20)
        except openai.error.APIError:
            print('openai.error.APIError\nRetrying...')
            time.sleep(20)
        except openai.error.APIConnectionError:
            print('openai.error.APIConnectionError\nRetrying...')
            time.sleep(20)
    print("gpt4:", res['choices'][0]['message']['content'])
    if "yes" in res['choices'][0]['message']['content'].lower():
        label = 1
    elif "no" in res['choices'][0]['message']['content'].lower():
        label = 0
    return label


def calculator(sentence, formula):
    try:
        result = sympify(formula)
        if isinstance(result, float):
            # 将result转换为字符串，以检查小数点后的位数
            result_str = str(result)
            # 检查是否存在小数点且小数点后的位数大于4
            if '.' in result_str and len(result_str.split('.')[1]) > 4:
                # 如果满足条件，保留4位小数
                result = round(result, 4)
        fact = f"{formula} = {result}"
        label = match(sentence, fact)
        return fact, label
    except SyntaxError as e:
        return f"Error: {e}"

def get_answer(sentences, labels, facts):
    sen = []
    for i in range(len(labels)):
        if labels[i] == 1:
            sen.append({"sentence":sentences[i], "fact":facts[i]})
    if len(sen) == 0:
        return "否。"
    else:
        return "是。" + str(sen)

def word_count(length, text):
    # 计算答案的字数
    answer_length = len(text)

    # 比较答案字数和要求字数，添加标签
    label = 0 if answer_length == length else 1

    return answer_length, label

def code_interpreter(code):
    print(code)
    an, report = safe_execute(code)
    print(an, report)
    label = 1
    if report == "Done":
        label = 0
    return label, report

def translate(text, target_language='en'):
    translator = Translator()
    result = translator.translate(text, dest=target_language)
    return result.text

def calculate_days_between_dates(date1, date2):
    date_format = "%Y-%m-%d"
    d1 = datetime.datetime.strptime(date1, date_format)
    d2 = datetime.datetime.strptime(date2, date_format)
    delta = d2 - d1
    return delta.days


In [ ]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

# Define helper functions
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, 'a+', encoding='utf-8') as f:
        json_record = json.dumps(data, ensure_ascii=False)
        f.write(json_record + '\n')

def init_model(model_path=None):
    """
    Initialize the model and tokenizer.
    """
    model = AutoModelForCausalLM.from_pretrained(
        "openai-community/gpt2",
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    model.generation_config = GenerationConfig.from_pretrained(
        "openai-community/gpt2"
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "openai-community/gpt2",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer

def get_response(messages):
    """
    Get a response from the model based on input messages.
    """
    model, tokenizer = init_model()
    response = model.chat(tokenizer, messages)
    return response

def generate_p(input_path, output_path):
    """
    Process the input JSON and generate output.
    """
    model, tokenizer = init_model()

    with open(input_path, "r", encoding="utf-8") as f:
        # Process each line as a separate JSON object
        for line in f:
            try:
                data = json.loads(line.strip())  # Strip any extra whitespace/newline
                # Process the data as before
                ...
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError: {e} - Line: {line.strip()}")
                continue  # Skip this line and move to the next

    cnt = 0
    res = []
    fail_num = []

    temp = "You are an agent tasked with detecting hallucinations in reply texts using a specific framework. Below is a detailed explanation of the detection framework:\nFirstly, you need to determine whether to split the input reply text into a list of sentences using a sentence segmentation tool. If required, you should check each sentence individually; otherwise, the entire text should be checked as a whole. You can choose an appropriate fact-checking tool to obtain relevant information and knowledge for verification and then use the matching tool to output the judgment results or directly output the judgment results. If you do not use the match tool and directly output the judgment results, you need to output the label in your thought. There is an error output \"label = 1\"; there is no error output \"label = 0\". After the verification is completed, you need to reflect on all detection results and output the label in your thought, then call get\_answer() to produce the final detection result. \n\nSentence Splitting Tool: \nsplit_text(text: str) → sentence_list\nThis function splits the text into a list of sentences.\n\nFact-Checking Tools: \nweb_search(sentence: str) → fact\nThis function uses a search engine to find information related to the sentence. After using web_search, you must use the match tool to determine if the reply matches the retrieved information.\n\ncalculator(sentence: str, formula: str) → result, label\nThis function uses a calculator to obtain the result of a formula and checks if the result matches the sentence. If they match, the label is 0; otherwise, it is 1. Valid operators include +, -, *, /, and parentheses. For instance, a valid input could be “(1 + 2) * 3”. If the input is an equation, it needs to be converted to a formula without unknowns.\n\nword_count(length: int, text: str) → count, label\nThis function calculates the word count of a text and outputs the count. If the word count does not meet the specified length, the label is 1; otherwise, it is 0.\n\ncode_interpreter() → label\nThis function checks whether the code can execute correctly. If it executes correctly, the output label is 0; otherwise, it is 1.\n\nMatching Tool:\nmatch(sentence: str, context:str) → label\nThis function checks a sentence against its context, which might include content from questions and replies around the detected sentence. It looks for irrelevant or contradictory answers. If any are found, the label is 1; otherwise, it is 0. If you think the output of match is wrong, you can correct the label in thought. For example, if you think the \"label = 0\" output by match is wrong, you can correct the answer and output \"label = 1\" in thought.\n\nEach time it’s your turn to respond, you must strictly follow this format to present your thoughts and actions: \"Thought: Your thought process.\nACTION: Tool call, e.g., match(sentence=\"...\", context=\"...\")\". After each tool use, I will provide the output as follows: \"Observation: Tool's output result\"."

    for d in data:
        try:
            cnt += 1
            fnum = 0
            messages = []
            query = {"role": "user", "content": temp + " QUERY: " + d["question"] + " RESPONSE: " + d["answer"]}
            messages.append(query)

            # Get the model's response
            ans = model.chat(tokenizer, messages)

            # Process the response to extract actions
            if "ACTION:" in ans:
                action = ans.split("ACTION:")[-1].strip(".").strip()
                if "\n" in action:
                    action = action.split("\n")[0]
                ans = ans.split("ACTION:")[0] + " ACTION: " + action

            # Variables for holding the results
            label_list = []
            fact_list = []
            sen_list = []
            web_fact = ""

            # Loop through actions and tools
            while True:
                response = ""
                response0 = "OBSERVATION:"
                action = ans.split("ACTION:")[-1].strip(".").strip(":").strip()
                print(action)

                if "split" in action:
                    sentences = eval(action)
                    response = {"role": "user", "content": response0 + str(sentences)}
                elif "calculator" in action:
                    result, label = eval(action)
                    response = {"role": "user", "content": response0 + result + ", label = " + str(label)}
                    sentence = action.split("sentence=")[-1].split("\"")[1]
                    sen_list.append(sentence)
                    fact_list.append(result)
                    label_list.append(label)
                elif "web_search" in action:
                    fact = eval(action)
                    response = {"role": "user", "content": response0 + fact}
                    web_fact = fact
                elif "match" in action:
                    label = eval(action)
                    text = "label = " + str(label)
                    response = {"role": "user", "content": response0 + text}
                    sentence = action.split("sentence=")[-1].split("\"")[1]
                    sen_list.append(sentence)
                    fact = action.split("context=")[-1].strip("\")")
                    fact_list.append(fact)
                    label_list.append(label)
                elif "code_interpreter" in action:
                    label, report = eval(action)
                    text = "label = " + str(label)
                    response = {"role": "user", "content": response0 + text}
                    sen_list.append(d["answer"])
                    fact_list.append(report)
                    label_list.append(label)
                elif "word_count" in action:
                    text = d["answer"]
                    count, label = eval(action)
                    response = {"role": "user", "content": response0 + "The number of words in this text is " + str(count) + ", label = " + str(label)}
                    sen_list.append(d["answer"])
                    fact_list.append("The number of words in this text is " + str(count))
                    label_list.append(label)
                elif "get_answer" in action:
                    fnum += 1
                    if fnum > 5:
                        break
                    if sen_list == []:
                        sen_list.append(d["answer"])
                        if web_fact:
                            fact_list.append(web_fact)
                    if label_list == []:
                        response = {"role": "user", "content": "OBSERVATION: The label is not detected, please output the label in THOUGHT like label = 1 or label = 0 and call get_answer() again."}
                        print(response)
                        messages.append(response)
                        ans = model.chat(tokenizer, messages)
                        if "ACTION:" in ans:
                            action = ans.split("ACTION:")[-1].strip(".").strip()
                            thought = ans.split("ACTION:")[0].strip(".").strip()
                            if "label=1" in thought.lower():
                                if len(label_list) > 0:
                                    label_list[-1] = 1
                                else:
                                    label_list.append(1)
                            elif "label=0" in thought.lower():
                                if len(label_list) > 0:
                                    label_list[-1] = 0
                                else:
                                    label_list.append(0)
                            if "\n" in action:
                                action = action.split("\n")[0]
                            ans = ans.split("ACTION:")[0] + " ACTION: " + action
                        messages.append({"role": "assistant", "content": ans})
                        print(ans)
                        continue
                    final_answer = get_answer(sen_list, label_list, fact_list)
                    messages.append({"role": "user", "content": "OBSERVATION: " + final_answer})
                    break
                else:
                    print("No such tool, failed!")
                    break

                # Add the response to the conversation
                messages.append(response)
                ans = model.chat(tokenizer, messages)
                if "ACTION:" in ans:
                    action = ans.split("ACTION:")[-1].strip(".").strip()
                    messages.append({"role": "assistant", "content": ans})
        except Exception as e:
            print(e)
            fail_num.append(cnt)
            d["final_answer"] = "Error"
            d["trajectory"] = messages
            res.append(d)
            dump_jsonl(d, output_path, append=True)
            continue

        d["final_answer"] = final_answer
        d["trajectory"] = messages
        res.append(d)
        dump_jsonl(d, output_path, append=True)

    print(f"{len(fail_num)} questions failed processing")
    print(fail_num)

# Specify the input and output file paths
input_path = "trajectory_generation_en.jsonl"  # Change to your uploaded input file name
output_path = "output_file.json"        # Change to your desired output file name



In [ ]:
from google.colab import files

# Upload the input file
uploaded = files.upload()

# List uploaded filesx
import os
print(os.listdir())


MessageError: RangeError: Maximum call stack size exceeded.

In [ ]:
# Call the main function
generate_p(input_path, output_path)

JSONDecodeError: Invalid \escape: line 1 column 983 (char 982) - Line: {"role": "system", "content": "You are an agent tasked with detecting hallucinations in reply texts using a specific framework. Below is a detailed explanation of the detection framework:\nFirstly, you need to determine whether to split the input reply text into a list of sentences using a sentence segmentation tool. If required, you should check each sentence individually; otherwise, the entire text should be checked as a whole. You can choose an appropriate fact-checking tool to obtain relevant information and knowledge for verification and then use the matching tool to output the judgment results or directly output the judgment results. If you do not use the match tool and directly output the judgment results, you need to output the label in your thought. There is an error output \"label = 1\"; there is no error output \"label = 0\". After the verification is completed, you need to reflect on all detection result

TypeError: 'str' object does not support item assignment

In [ ]:

-


Colab is used extensively in the machine learning community with applications including:
- Getting started with TensorFlow
- Developing and training neural networks
- Experimenting with TPUs
- Disseminating AI research
- Creating tutorials

To see sample Colab notebooks that demonstrate machine learning applications, see the [machine learning examples](#machine-learning-examples) below.

<div class="markdown-google-sans">

## More Resources

### Working with Notebooks in Colab

</div>

- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<div class="markdown-google-sans">

<a name="working-with-data"></a>
### Working with Data
</div>

- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

<div class="markdown-google-sans">

### Machine Learning Crash Course

<div>

These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Intro to RAPIDS cuDF to accelerate pandas](https://nvda.ws/rapids-cudf)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)

<div class="markdown-google-sans">

<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
</div>

- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

<div class="markdown-google-sans">

<a name="machine-learning-examples"></a>

### Featured examples

</div>

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.
